

# Credo AI Connect

**Credo AI Connect** is the library to interact with the Credo AI Governance Platform. It is responsible for receiving "evidence requirements" and sending "evidence"

In this demo we train a model on some made-up data, and send the results to the [Credo AI Governance Platform](https://www.credo.ai/). For a tool to help you assess your AI systems, see [Credo AI Lens](https://credoai-lens.readthedocs.io/en/stable/setup.html).

**Setup**

Connect installation instruction can be found on [readthedocs](https://credoai-connect.readthedocs.io/en/stable/setup.html)

**Find the code**

## Useful keywords

- **Credo AI Platform**: Also referred to as simply "Platform". The central AI governance/reporting Platform, found at [https://app.credo.ai/](https://app.credo.ai/)

- **credoconfig**: configuration file to be copied in the user's home folder

- **use_case_name**: The name of your Use Case as it is registered on Credo AI Platform

- **policy_pack**: A set of governance controls that a Use Case needs to satisfy. A Use Case can have multiple policy packs applied to it.

- **policy_pack_key**: A unique identifier for a policy pack (where do we get this?)

- **assessment_plan_url**: The link to the assessment plan, this is generated in the Platform and used to download the assessment plan in the Governance object. See example below.

- **evidence**: Any evaluation of an AI system, formatted specifically to be uploaded to the platform.


## Setup API Connection with the Platform

### Get a config file
This file contains all the necessary information to connect Lens to the Credo AI Platform.  

To generate the config file, once you logged into the platform, click on your name icon (top left) and follow:  

`My Settings -> Tokens -> Plus Sign -> Generate`

Immediately after generating the token, you will be given the possibility to download the config file.

The default location/file name Lens is expecting is `~/.credoconfig`, where `~` is your home folder. You can specify any other location when you are initializing the `Governance` object (see below).

## Do some AI stuff!

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
import numpy as np

### Get data and train model

We'll make up some data and fit a simple model to get us started.

In [ ]:
model = LinearRegression()
r = np.random.RandomState(42)

# hallucinate data
N = 10000
data = r.randn(N, 2)
y = (data[:,0] + 3*data[:,1] + r.randn(N)*.2)
# split
X_train, X_test, y_train, y_test = train_test_split(data, y)

# train model
model.fit(X_train, y_train)
pred_test = model.predict(X_test)

### Assess the AI System

The most likely thing you'll want to send to the Credo AI Platform are _assessments_ of the AI system. These are often _metrics_ that are calculated to summarize aspects of the system's behavior (e.g., performance, fairness) but can be other things too, like descriptive statistics of the dataset.

We'll assume we have a suite of assessments we plan to use for this use-case. The below is just an example - any assessments can be done and sent to the Platform.

In [ ]:
required_assessments = [sk_metrics.r2_score, sk_metrics.mean_squared_error]

assessments = {m.__name__: m(y_test, pred_test) for m in required_assessments}

## Platform integration in 5 Minutes

To send evidence to the platform takes three steps:

1. Connect to the platform via the Governance class
2. Use the Adapter class to send the assessments to the Governance class
3. Export the evidence to the platform.

First we will see all the code together, and then break it down.

In [ ]:
import connect as ct

In [ ]:
# connect to platform via governace
gov = ct.Governance()  # Specify config_path if your config file is not the default one: ~/.credoconfig
url = 'your assessment url'
gov.register(assessment_plan_url=url)

# set up adapter and send metrics to governance class
adapter = ct.Adapter(governance = gov, model_name='My Model')
source = f"Quickstart_Connect-{ct.__version__}"
adapter.metrics_to_governance(metrics=assessments, source = source)

# export
gov.export()

#### 1. Connect to the platform via the Governance Class

The Governance class handles the connection with the Credo AI Platform. On the Platform, you can govern an AI system by specifying "policy packs" that specify the technical requirements the AI system must meet. This class can retrieve them, which is most useful if you are using [Credo AI Lens](https://credoai-lens.readthedocs.io/en/stable/setup.html), our assessment framework. If you aren't using Lens, the requirements are still important in directing your assessment, but they aren't _programatically_ connected to your assessments.

Since we are only using `Connect` in this demo, we will ignore that functionality.

The important functionality relevant here is that the Governance object handles the API calls allowing you to _send_ evidence to the platform. 

In [ ]:
# Retrieve Policy Pack Assessment Plan
gov = ct.Governance()  # Specify config_path if your config file is not the default one: ~/.credoconfig
url = 'your assessment url'
gov.register(assessment_plan_url=url)

#### 2. Use the Adapter class to send the assessments to the Governance class

The `Adapter` class handles structuring your assessments so that the Credo AI Platform can understand them. Connect uses `EvidenceContainers` to handle converting python objects like dictionaries into `Evidence`, the structured output that the Platform can understand. The Adapter then passes this `Evidence` to the `Governance` object for export

In [ ]:
adapter = ct.Adapter(governance = gov, model_name='My Model')


Once we initialize an adapter, we can use its functionality to send different kinds of evidence.

For instance, metrics (which must be organized as key:value pairs of metric_name:value) are sent to governance using `metrics_to_governance`
This converts the dictionary of assessments into `Evidence` (in this case a `MetricEvidence`).

Every time we send evidence we specify a source. This is for you to define however you would like. We suggest you make the source useful to establish provenance of your assessments. 

In [ ]:
source = f"Quickstart_Connect-{ct.__version__}"
adapter.metrics_to_governance(metrics=assessments, source = source)

You can see the evidence in the `Governance` class we instantiated before

In [ ]:
gov.get_evidence()

If you need to send more evidence, you'll just call a function like `metrics_to_governance` again. By default, this functions overwright the `Evidence` in `Governance` so you'll have to change an argument to allow overwriting.

We will send a different type of evidence - a table, which must be a pandas DataFrame.

In [ ]:
import pandas as pd
table = pd.DataFrame({'arbitrary_data': [3,4,5]})
table.name = 'my_table'
adapter.table_to_governance(table, source=source, overwrite_governance=False)

In [ ]:
gov.get_evidence()

#### 3. Export the evidence to the platform.

Exporting is straight forward! You can either export directly to the Platform or to a file. At the time of export, the uploaded evidence will be checked against the governance requirements specified on Platform and let you know what's missing.

In [ ]:
# export to API
gov.export()

# export to file
gov.export("assessment_export.json")

## Other ways to label your evidence

On the Credo AI Platform, certain governance requirements may only apply to models that are tagged in a certain way. You can get the requirements tags from governance and apply it by passing a dictionary to the `model_tags` of the `Adapter`

In [ ]:
gov.get_requirement_tags()

In [ ]:
adapter = ct.Adapter(governance = gov, model_name='My Model', model_tags={'model_type': 'regression'})

If you need to label your evidence with more information than just `source` you can do that too.

## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model. We'll break down this code [below](#Breaking-Down-The-Steps).

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [ ]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

# initialize the evaluator and add it to Lens
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))

# run Lens
lens.run()

### Getting results within your python environment

`lens.get_results()` provides a list where the results of the evaluators (a list of dataframes) are stored along with the evaluator metadata. In this case, there are 2 results - one for each evaluator.

In [ ]:
results = lens.get_results()
print(f"Results for {len(results)} evaluators")

`lens.get_results()` has some arguments, which makes it easier for you to get a subset of results. These are the same arguments that can be passed to `lens.get_pipeline` and `lens.get_evidence`

In [ ]:
lens.get_results(evaluator_name='Performance')

## Using Len's pipeline argument

If we want to add multiple evaluators to our pipeline, one way of doing it could be repeating the `add` step, as shown above. Another way is to define the pipeline steps, and pass it to `Lens` at initialization time. Let's explore the latter!

In [ ]:
pipeline = [
    (ModelFairness(metrics)),
    (Performance(metrics)),
]
lens = Lens(model=credo_model, assessment_data=credo_data, pipeline=pipeline)

Above, each of the `tuples` in the `list` is in the form `(instantiated_evaluator, id)`.

In [ ]:
# notice that Lens functions can be chained together
results = lens.run().get_results()
print(f'\nFound results for {len(results)} evaluators')

Let's check that we have results for both of our evaluators.

In [ ]:
results[0]

In [ ]:
results[1]

## That's it!

That should get you up and running. Next steps include:

* Trying out other evaluators (they are all accessible via `credoai.evaluators`)
* Checking out our developer guide to better understand the Lens ecosystem and see how you can extend it.
* Exploring the Credo AI Governance Platform, which will connect AI assessments with customizable governance to support reporting, compliance, multi-stakeholder translation and more!

## Breaking Down The Steps

### Preparing artifacts

Lens interacts with "AI Artifacts" which wrap model and data objects and standardize them for use by different evaluators.

Below we create a `ClassificationModel` artifact. This is a light wrapper for any kind of fitted classification model-like object. 

We also create a `TabularData` artifact which stores X, y and sensitive features.

In [ ]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)

credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

#### Model

Model type objects, like `ClassificationModel` used above, serve as adapters between arbitrary models and the evaluators in Lens. Some evaluators depend on Model instantiating certain methods. For example, `ClassificationModel` can accept any generic object having `predict` and `predict_proba` methods, including fitted sklearn pipelines.



#### Data

_Data_ type artifact, like `TabularData` serve as adapters between datasets and the evaluators in Lens.

When you pass data to a _Data_ artifact, the artifact performs various steps of validation, and formats them so that they can be used by evaluators. The aim of this procedure is to preempt errors down the line.

You can pass Data to Lens as a **training dataset** or an **assessment dataset** (see lens class documentation). If the former, it will not be used to assess the model. Instead, dataset assessments will be performed on the dataset (e.g., fairness assessment). The validation dataset will be assessed in the same way, but _also_ used to assess the model, if provided.

Similarly to _Model_ type objects, _Data_ objects can be customized, see !!insertlink!!

### Evaluators 

Lens uses the above artifacts to ensure a successfull run of the evaluators. As we have seen in the sections [Lens in 5 minutes](##Lens-in-5-minutes) and [Adding multiple evaluators](##Adding-multiple-evaluators), multiple evaluators can be added to _Lens_ pipeline. Each evaluators contains information on what it needs in order to run successfully, and it executes a validation step at _add_ time.

The result of the validation depends on what artifacts are available, their content and the type of evaluator being added to the pipeline. In case the validation process fails, the user is notified the reason why the evaluator cannot be added to the pipeline.

See for example:

In [ ]:
from credoai.evaluators import Privacy
lens.add(Privacy())

Currently no automatic run of evaluators is supported. However, when Lens is used in combination with Credo AI Platform, it is possible to download an assessment plan which then gets converted into a set of evaluations that Lens can run programmatically. For more information see the [governance tutorial](https://credoai-lens.readthedocs.io/en/stable/notebooks/governance_integration.html).

### Run Lens

After we have initialized _Lens_ the _Model_ and _Data_ (`ClassificationModel` and `TabularData` in our example) type artifacts, we can add whichever evaluators we want to the pipeline, and finally run it!

In [ ]:
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.run()

As you can notice, when adding _evaluators_ to lens, they need to be instantiated. If any extra arguments need to be passed to the evaluator (like metrics in this case), this is the time to do it.

**Getting Evaluator Results**

Afte the pipeline is run, the results become accessible via the method `get_results()`

`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

In [ ]:
lens.get_results()

**Credo AI Governance Platform**

For information on how to interact with the plaform, please look into: [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/stable/notebooks/governance_integration.html) tutorial for directions.
